In [1]:
import torch

In [38]:
names = [name.strip() for name in open('names.txt','r').readlines()]
names = ['!'+name+'!' for name in names]

In [39]:
# trigrams = {char: [] for char in list(set((letter for name in names for letter in name)))}
trigrams={}
for name in names:
    for i in range(len(name)-2):
        trigrams[name[i]+name[i+1]]=[]

In [40]:
for name in names:
    for i in range(len(name)-2):
        # if name[i]+name[i+1]=='jp':
        #     print(name[i+2])
        trigrams[name[i:i+2]].append(name[i+2])


In [50]:
import numpy as np
unique_chars = sorted(list(set((letter for name in names for letter in name))))
prob = np.zeros((len(unique_chars)**2,len(unique_chars)),dtype=np.float16)
conversion_1 = {char: i for i, char in enumerate(unique_chars)}
convBack_1 = {i: char for i, char in enumerate(unique_chars)}
combs = [i+j for i in unique_chars for j in unique_chars[1:]]
conversion_2 = {comb: i for i, comb in enumerate(combs)}
convBack_2 = {i: comb for i, comb in enumerate(combs)}
for key, values in trigrams.items():
    for value in values:
        prob[conversion_2[key],conversion_1[value]]+=1


In [51]:
prob /= np.sum(prob,axis=1)[:,None]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18968\3883716547.py:1: RuntimeWarning: invalid value encountered in divide
  prob /= np.sum(prob,axis=1)[:,None]


In [52]:
def generate_name():
    name = '!'+ np.random.choice(unique_chars)
    while name[-1] != '!':
        name += convBack_1[np.random.choice(len(unique_chars),p=prob[conversion_2[name[-2:]]])]
    return name

In [69]:
generate_name()[1:-1]

'sabea'

In [71]:
X,y = [],[] 
for key in trigrams.keys():
    for value in trigrams[key]:
        feature = np.zeros((len(combs)))
        feature[conversion_2[key]]=1
        X.append(feature)
        y.append(conversion_1[value])


In [72]:
X,y = np.array(X),np.array(y)
idx = np.random.permutation(len(X))
X,y = X[idx],y[idx]
X,y = torch.tensor(X, dtype=torch.float32),torch.tensor(y, dtype=torch.int32)

In [75]:
#for reproducibility
gen = torch.Generator().manual_seed(69)
W = torch.randn((len(combs),len(unique_chars)), generator=gen, requires_grad=True)


In [76]:
rate = 30
for i in range(1000):
    logits = X @ W
    probs = torch.nn.functional.softmax(logits,dim=1)
    loss = -torch.mean(probs[torch.arange(len(y)),y].log())
    loss.backward()
    with torch.no_grad():
        W -= rate * W.grad
        W.grad.zero_()

    if i % 100 == 0:
        print(loss)
        rate *= 0.9

tensor(3.7307, grad_fn=<NegBackward0>)
tensor(2.5552, grad_fn=<NegBackward0>)
tensor(2.3888, grad_fn=<NegBackward0>)
tensor(2.3151, grad_fn=<NegBackward0>)
tensor(2.2732, grad_fn=<NegBackward0>)
tensor(2.2462, grad_fn=<NegBackward0>)
tensor(2.2274, grad_fn=<NegBackward0>)
tensor(2.2136, grad_fn=<NegBackward0>)
tensor(2.2030, grad_fn=<NegBackward0>)
tensor(2.1948, grad_fn=<NegBackward0>)


In [95]:
def generate_name_nn():
    name = '!'+ np.random.choice(unique_chars)
    while name[-1] != '!':
        feature = np.zeros((len(combs)))
        feature[conversion_2[name[-2:]]]=1
        feature = torch.tensor(feature, dtype=torch.float32)
        logits = feature @ W
        probs = torch.nn.functional.softmax(logits, dim=0)
        ToAdd = torch.multinomial(probs,1).item()
        name += convBack_1[ToAdd]
    return name[1:-1]

In [98]:
generate_name_nn()

'gabkjznle'